In [1]:
from bs4 import BeautifulSoup
import requests, json
import pandas as pd
import re
import time
import os
import collections


pd.options.mode.chained_assignment = None

In [44]:

HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}
url = "https://auto.ru/-/ajax/desktop/listing/"

#url = 'https://auto.ru/-/ajax/desktop/listingSpecial/'

In [3]:
test = pd.read_csv('test.csv')

bmw_test = test.rename(columns = {'Комплектация': 'complectation', 'Привод': 'lk_summary', 
                                  'Руль': 'steering_wheel', 'Состояние': 'steering_wheel',
                                  'Владельцы': 'owners_number', 'ПТС': 'pts', 
                                  'Таможня': 'custom_cleared', 'Владение': 'purchase_date'}, inplace = False)
# посмотрим какие столбцы нам нужны 
print('Кол-во столбцов теста:', bmw_test.shape[1])
print('Названия столбцов теста:', list(bmw_test.columns))

# вывод всех столбцов датафрейма
pd.set_option('display.max_columns', None)
bmw_test.head(2)

Кол-во столбцов теста: 23
Названия столбцов теста: ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'engineDisplacement', 'enginePower', 'description', 'mileage', 'complectation', 'lk_summary', 'steering_wheel', 'steering_wheel', 'owners_number', 'pts', 'custom_cleared', 'purchase_date', 'id']


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,complectation,lk_summary,steering_wheel,steering_wheel,owners_number,pts,custom_cleared,purchase_date,id
0,седан,BMW,чёрный,дизель,2016.0,520d 2.0d AT (190 л.с.),4.0,2017.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,190 N12,В РОЛЬФ Ясенево представлено более 500 автомоб...,158836.0,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,0
1,седан,BMW,белый,дизель,2018.0,318d 2.0d AT (150 л.с.),4.0,2019.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,150 N12,Автомобиль из демонстрационного парка по спец ...,10.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,1


In [57]:
# но сначала установим главные параметры запроса
mark = 'BMW'  # марка
#model = 'M6'  # модель
geo_radius = 300  # радиус вокруг москвы = 200 км
geo_id = [213]  # Москва
top_days = "900"  # кол-во дней от текущей даты назад за которые запрашиваем объявления 


offers = []
for num_page in range(1, 200):
    param = {
        "catalog_filter" : [{"mark": mark}],
        #"catalog_filter" : "all",
        "section": "all",
        "category": "cars",
        "sort": "fresh_relevance_1-desc",
        "top_days": top_days,
        "geo_radius":geo_radius,
        "geo_id":geo_id,
        "page": num_page
        }
    response = requests.post(url, json=param, headers=HEADERS)
    data = response.json()
    offers.extend(data['offers'])
    
    

In [63]:
len(offers)

7475

In [65]:
with open('bmw_json.json', 'w') as f:
    json.dump(offers, f)
    

In [66]:
with open('bmw_json.json', 'r', encoding='utf8') as f:
    bmw_json = json.load(f)

In [67]:
len(bmw_json)

7475

In [68]:
#Let's create dictionary for our function
dict_func = {}

# создаем функцию для обработки 23 необходимых столбцов
def parse_to_df(bmw_json, dict_func):
    data = pd.DataFrame()
    for offers in bmw_json:
    
        try: dict_func['bodyType'] = offers['vehicle_info']['configuration']['human_name']
        except: dict_func['bodyType'] = None

        try: dict_func['brand'] = offers['vehicle_info']['mark_info']['code']
        except: dict_func['brand'] = None

        try: dict_func['color'] = offers['color_hex']
        except: dict_func['color'] = None

        try: dict_func['fuelType'] = offers['lk_summary'].split()[-1]
        except: dict_func['fuelType'] = None

        try: dict_func['modelDate'] = offers['vehicle_info']['super_gen']['year_from']
        except: dict_func['modelDate'] = None

        try: dict_func['name'] =  offers['vehicle_info']['tech_param']['human_name']
        except: dict_func['name'] = None

        try: dict_func['numberOfDoors'] = offers['vehicle_info']['configuration']['doors_count']
        except: dict_func['numberOfDoors'] = None

        try: dict_func['productionDate'] = offers['documents']['year']
        except: dict_func['productionDate'] = None

        try: dict_func['vehicleTransmission'] = offers['vehicle_info']['tech_param']['transmission']
        except: dict_func['vehicleTransmission'] = None

        try: dict_func['engineDisplacement'] = offers['vehicle_info']['tech_param']['human_name'].split()[0]
        except: dict_func['engineDisplacement'] = None

        try: dict_func['vehicleConfiguration'] = offers['vehicle_info']['configuration']['body_type']+' '+d_dict['vehicleTransmission']+' '+d_dict['engineDisplacement']
        except: dict_func['vehicleConfiguration'] = None

        try: dict_func['enginePower'] = offers['vehicle_info']['tech_param']['power']
        except: dict_func['enginePower'] = None

        try: dict_func['description'] = offers['description']
        except: dict_func['description'] = None

        try: dict_func['mileage'] = offers['state']['mileage']
        except: dict_func['mileage'] = None

        try: dict_func['complectation'] = offers['vehicle_info']['complectation']
        except: dict_func['complectation'] = None

        try: dict_func['lk_summary'] = offers['lk_summary'].split(', ')[-2]
        except: dict_func['lk_summary'] = None

        try: dict_func['steering_wheel'] = offers['vehicle_info']['steering_wheel']
        except: dict_func['steering_wheel'] = None

        # 'Состояние' не найдено
        dict_func['Condition'] = None

        try: dict_func['owners_number'] = offers['documents']['owners_number']
        except: dict_func['owners_number'] = None

        try: dict_func['pts'] = offers['documents']['pts']
        except: dict_func['pts'] = None

        try: dict_func['custom_cleared'] = offers['documents']['custom_cleared']
        except: dict_func['custom_cleared'] = None
        
        try: dict_func['purchase_date'] =  offers['documents']['purchase_date']
        except: dict_func['purchase_date'] = None

        try: dict_func['price'] = offers['price_info']['RUR']
        except: dict_func['price'] = None

        # дата размещения объявления решил добавить 
        try: dict_func['start_date'] =  offers['additional_info']['hot_info']['start_time']
        except: dict_func['start_date'] = None

        # статус объявления 
        try: dict_func['hidden'] =  offers['additional_info']['hidden']
        except: dict_func['hidden'] = None

        # модель 
        try: dict_func['model'] = offers['vehicle_info']['model_info']['code']
        except: dict_func['model'] = None

        data = data.append(dict_func, ignore_index=True)

    return data

In [69]:
bmw_df = parse_to_df(bmw_json, dict_func)
bmw_df.head(2)
bmw_df.shape

(7475, 26)

In [71]:
bmw_df['brand'].value_counts()

BMW    7475
Name: brand, dtype: int64

In [72]:
bmw_df.to_csv('bmw_df.csv')

In [ ]:
bmw_test.head(2)

In [ ]:
import utils_module09092020 as utils